<a href="https://colab.research.google.com/github/SaveP242/Transformers-NLP/blob/main/recipegeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.0 MB/s eta 0:00:00


In [ ]:
import torch
import numpy
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load and preprocess your recipe dataset
dataset_file = "dataset.txt"
with open(dataset_file, "r") as file:
    recipes = file.read().split("\n")

# Tokenize and format the data
input_ids = tokenizer.encode(" ".join(recipes), return_tensors='pt', max_length=1024, truncation=True)

In [ ]:
# Create a dataset and data collator
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_file,
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./recipe_finetuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=10_000,
    logging_steps=1_000,
    logging_dir="./logs",
    learning_rate=1e-4,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=False,  # Disabling load_best_model_at_end
    push_to_hub=False,
)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,

)

# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./recipe_finetuned_model")
tokenizer.save_pretrained("./recipe_finetuned_model")

In [ ]:
# Generate recipes using the fine-tuned model
input_text = "Generate a recipe for a cake"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

output = model.generate(input_ids, max_length=200, num_return_sequences=1)

# Decode and print the generated recipe
generated_recipe = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_recipe)